In [1]:
from datasets import load_dataset
from google import genai
from tenacity import retry, wait_exponential, stop_after_attempt
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

/opt/anaconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
dataset = load_dataset("sentence-transformers/stsb", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")


Loaded 5749 examples. Sample:
{'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.', 'score': 1.0} 



In [3]:
# Set your OpenAI API key (best via environment variable)
client = genai.Client(api_key=os.getenv("GOOGLE_GENERATIVE_AI_API_KEY")) 

In [4]:
# Function to get embedding
@retry(wait=wait_exponential(multiplier=1, min=4, max=60), stop=stop_after_attempt(5))
def get_embedding(text, model_name="gemini-embedding-001"):
    try:
        response = client.models.embed_content(
            model = model_name,
            contents=text
        )

        return response.embeddings
    
    except Exception as e:
        print(f"Error for text: {text[:30]}... => {e}")
        return None

In [6]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
model_name = 'gemini-embedding-001'
file_to_save = 'gemini_text-'+model_name+'.pickle'
embeddings = pd.DataFrame()
sentence1 = []
sentence2 = []
embedding1 = []
embedding2 = []
cosine_similarity = []
human_score = []

print('Extracting embedding and measuring similarity ...')
time.sleep(1)
for idx in tqdm(range(len(dataset))):
    first_sentence = dataset[idx]["sentence1"]
    emb1 = list(get_embedding(first_sentence, model_name)[0])[0][1]
    sentence1.append(
        first_sentence
    )
    embedding1.append(
        emb1
    )
    #############################
    second_sentence = dataset[idx]["sentence2"]
    emb2 = list(get_embedding(second_sentence, model_name)[0])[0][1]
    sentence2.append(
        second_sentence
    )
    embedding2.append(
        emb2
    )
    
    ############################
    cosine_similarity.append(
        1 - cosine(emb1, emb2)
    )
    human_score.append(
        dataset[idx]["score"]
    )
    
    time.sleep(1)  # respect rate limits!

embeddings['Sentence 1'] = sentence1
embeddings['Sentence 2'] = sentence2
embeddings['Embedding of Sentence 1'] = embedding1
embeddings['Embedding of Sentence 2'] = embedding2
embeddings['Cosine similarity'] = cosine_similarity
embeddings['Human score'] = human_score

with open(file_to_save, 'wb') as f:
    pickle.dump(embeddings, f)

Extracting embedding and measuring similarity ...


100%|██████████| 5749/5749 [2:26:25<00:00,  1.53s/it]  


In [7]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,A plane is taking off.,An air plane is taking off.,"[-0.03038812, 0.028442184, 0.0040001837, -0.08...","[-0.027468763, 0.037402656, 0.014797646, -0.07...",0.960455,1.00
1,A man is playing a large flute.,A man is playing a flute.,"[-0.02655112, -0.00027439563, 0.008827717, -0....","[-0.038136486, 0.01199731, 0.013802515, -0.111...",0.915832,0.76
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,"[-0.038540676, 0.015014919, 0.027023548, -0.03...","[-0.031529076, 0.017592212, 0.0175504, -0.0346...",0.960580,0.76
3,Three men are playing chess.,Two men are playing chess.,"[-0.009992072, -0.009598609, -0.0011456223, -0...","[-0.030862274, 0.006396856, -0.0153435385, -0....",0.885713,0.52
4,A man is playing the cello.,A man seated is playing the cello.,"[-0.04253136, 0.019396903, -0.008048837, -0.09...","[-0.037090085, 0.028250685, -0.0085480325, -0....",0.937368,0.85
